# BERT Base 
Uncased Embeddings

Using bert uncased embeddings to find a representative word embedding of trivial and non-trivial messages (from the training sentences). Classify each new sentence based on the Euclidean distance from the representative embeddings.

In [2]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf

from scipy.spatial.distance import cosine

2023-04-29 14:44:12.621521: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
checkmate_messages_df = pd.read_csv('../../src/data/CheckMate_Messages_Table.csv')

In [5]:
# Prepping data
checkmate_messages_df = pd.read_csv('../../src/data/CheckMate_Messages_Table.csv')
checkmate_messages_df.dropna()
checkmate_messages_df['is_trivial'] = (checkmate_messages_df['taggedCategory']=='Trivial')
# Not all sentences are imported as strings
checkmate_messages_df['text'] = checkmate_messages_df['text'].astype('str')

In [6]:
checkmate_messages_df['text'] = (checkmate_messages_df['text'].apply(lambda x: "[CLS] " + x + " [SEP]"))
print(checkmate_messages_df)
checkmate_messages_df['token_ids'] = checkmate_messages_df['text'].apply(lambda x: tokenizer.encode(x, truncation=True))
print(checkmate_messages_df)

                                                 text     taggedCategory  \
0                                     [CLS] nan [SEP]  Info/News/Opinion   
1   [CLS] https://www.mas.gov.sg/news/media-releas...  Info/News/Opinion   
2   [CLS] TN 95546718362782 is out for del. Allow ...         Legitimate   
3   [CLS] 🚩🚩🚩 *"You flag, we check"* 🔍🔍🔍\n\nNot su...  Info/News/Opinion   
4   [CLS] https://form.gov.sg/63f594b42413ea001183...         Legitimate   
..                                                ...                ...   
81  [CLS] [SHIN MIN CONTEST] Happycall Jumbo 双面锅等你...             Unsure   
82  [CLS] Hello, sorry to bother you, I'm Nico fro...               Scam   
83  [CLS] Hello, my name is Sarah, from CME Group....               Scam   
84  [CLS] LTA: Notice As no valid E-tag detected i...               Scam   
85  [CLS] Excuse me, this is Stella, have you arra...             Unsure   

     type  isMachineCategorised  truthScore  is_trivial  
0   image                 Fal

In [7]:
length = checkmate_messages_df['token_ids'].apply(lambda x: x)
for i in length:
    print(len(i))

5
46
42
336
30
55
9
27
5
49
72
15
42
97
141
50
59
32
111
69
124
28
275
29
254
89
512
47
281
449
512
67
42
70
324
81
381
14
12
118
7
15
5
9
132
8
9
8
5
89
47
36
40
63
5
50
6
113
46
7
63
9
7
21
32
68
50
117
37
6
5
69
33
5
87
69
63
112
26
8
26
112
145
40
46
26


In [8]:

checkmate_messages_df['segment_ids'] = (checkmate_messages_df['token_ids'].apply(lambda x: [1] * len(x)))


In [9]:
print(checkmate_messages_df)

                                                 text     taggedCategory  \
0                                     [CLS] nan [SEP]  Info/News/Opinion   
1   [CLS] https://www.mas.gov.sg/news/media-releas...  Info/News/Opinion   
2   [CLS] TN 95546718362782 is out for del. Allow ...         Legitimate   
3   [CLS] 🚩🚩🚩 *"You flag, we check"* 🔍🔍🔍\n\nNot su...  Info/News/Opinion   
4   [CLS] https://form.gov.sg/63f594b42413ea001183...         Legitimate   
..                                                ...                ...   
81  [CLS] [SHIN MIN CONTEST] Happycall Jumbo 双面锅等你...             Unsure   
82  [CLS] Hello, sorry to bother you, I'm Nico fro...               Scam   
83  [CLS] Hello, my name is Sarah, from CME Group....               Scam   
84  [CLS] LTA: Notice As no valid E-tag detected i...               Scam   
85  [CLS] Excuse me, this is Stella, have you arra...             Unsure   

     type  isMachineCategorised  truthScore  is_trivial  \
0   image                 Fa

In [10]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [11]:
checkmate_messages_df['tensor_token'] = checkmate_messages_df.apply(lambda x: torch.tensor([x['token_ids']]), axis=1)
checkmate_messages_df['tensor_segment'] = checkmate_messages_df.apply(lambda x: torch.tensor([x['segment_ids']]), axis=1)


In [12]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    checkmate_messages_df['hidden_states'] = checkmate_messages_df.apply(lambda x: model(x['tensor_token'],x['tensor_segment'])[2], axis=1)


In [13]:
checkmate_messages_df['hidden_states'].apply(lambda x: print(x[-2][0].shape))
# Checking that the size is correct

torch.Size([5, 768])
torch.Size([46, 768])
torch.Size([42, 768])
torch.Size([336, 768])
torch.Size([30, 768])
torch.Size([55, 768])
torch.Size([9, 768])
torch.Size([27, 768])
torch.Size([5, 768])
torch.Size([49, 768])
torch.Size([72, 768])
torch.Size([15, 768])
torch.Size([42, 768])
torch.Size([97, 768])
torch.Size([141, 768])
torch.Size([50, 768])
torch.Size([59, 768])
torch.Size([32, 768])
torch.Size([111, 768])
torch.Size([69, 768])
torch.Size([124, 768])
torch.Size([28, 768])
torch.Size([275, 768])
torch.Size([29, 768])
torch.Size([254, 768])
torch.Size([89, 768])
torch.Size([512, 768])
torch.Size([47, 768])
torch.Size([281, 768])
torch.Size([449, 768])
torch.Size([512, 768])
torch.Size([67, 768])
torch.Size([42, 768])
torch.Size([70, 768])
torch.Size([324, 768])
torch.Size([81, 768])
torch.Size([381, 768])
torch.Size([14, 768])
torch.Size([12, 768])
torch.Size([118, 768])
torch.Size([7, 768])
torch.Size([15, 768])
torch.Size([5, 768])
torch.Size([9, 768])
torch.Size([132, 768])
to

0     None
1     None
2     None
3     None
4     None
      ... 
81    None
82    None
83    None
84    None
85    None
Name: hidden_states, Length: 86, dtype: object

In [14]:
checkmate_messages_df['sentence_embedding'] = checkmate_messages_df['hidden_states'].apply(lambda x: torch.mean(x[-2][0], dim=0))

In [15]:
df_train, df_test= train_test_split(
        checkmate_messages_df, test_size=0.50, random_state=0, stratify=checkmate_messages_df['is_trivial'])

In [16]:
# average the sentence embeddings of non trivial and trivial messages
df_train_not_trivial = df_train[df_train['is_trivial'] == False]
df_train_trivial = df_train[df_train['is_trivial'] == True]

In [17]:
tensor_train_not_trivial = torch.stack([t for t in df_train_not_trivial['sentence_embedding']])
tensor_train_trivial = torch.stack([t for t in df_train_trivial['sentence_embedding']])


In [18]:
rep_not_trivial = torch.mean(tensor_train_not_trivial, dim=0)
rep_trivial = torch.mean(tensor_train_trivial, dim=0)

In [19]:
def classify_trivial(v):
    distance_from_not_trivial = cosine(rep_not_trivial, v)
    distance_from_trivial = cosine(rep_trivial, v)
    if distance_from_trivial > distance_from_not_trivial:
        return False
    else: 
        return True

In [20]:
f= df_test['sentence_embedding']
print(f[1].shape)

torch.Size([768])


In [21]:
df_test['prediction'] = df_test['sentence_embedding'].apply(lambda x: classify_trivial(x))

In [22]:
df_test.keys()

Index(['text', 'taggedCategory', 'type', 'isMachineCategorised', 'truthScore',
       'is_trivial', 'token_ids', 'segment_ids', 'tensor_token',
       'tensor_segment', 'hidden_states', 'sentence_embedding', 'prediction'],
      dtype='object')

In [32]:
predicted_vs_output = pd.concat([df_test['text'], df_test['is_trivial'], df_test['prediction'], df_test['taggedCategory'],df_test['text']], axis=1)
# predicted_vs_output = predicted_vs_output.rename(columns={'is_trivial':'target', 'text':'prediction'})
predicted_vs_output.columns = ['text', 'target','prediction','category']

print(predicted_vs_output)

                                                 text  target  prediction  \
11  [CLS] Hello, no matter how busy work remember ...    True        True   
15  [CLS] https://www.astroawani.com/berita-malays...   False       False   
78  [CLS] Hello, I just received a call from an un...    True       False   
71  [CLS] Dear Delegate, a gentle reminder that SM...   False       False   
41  [CLS] Hello \nsorry to bother you, is this Kev...   False        True   
64  [CLS] Noted. Sorry about this. I misread your ...    True       False   
76  [CLS] FYI, please. Please kindly adhere to the...    True       False   
44  [CLS] Hi , this is Julia from Chan Brothers Tr...   False       False   
24  [CLS] [8/04, today's Zaobao Chinese Morning ne...   False       False   
1   [CLS] https://www.mas.gov.sg/news/media-releas...   False       False   
81  [CLS] [SHIN MIN CONTEST] Happycall Jumbo 双面锅等你...   False       False   
45                       [CLS] Hello a bit slow [SEP]    True        True   

In [33]:
# Calculate the recall of this methodology
true_positive = len(predicted_vs_output[(predicted_vs_output['target']==True) & (predicted_vs_output['prediction']==True)])
true_negative = len(predicted_vs_output[(predicted_vs_output['target']==False) & (predicted_vs_output['prediction']==False)])

false_negative = len(predicted_vs_output[(predicted_vs_output['target']==True) & (predicted_vs_output['prediction']==False)])
false_positive = len(predicted_vs_output[(predicted_vs_output['target']==False) & (predicted_vs_output['prediction']==True)])

recall = true_positive/(true_positive+false_negative)
accuracy = (true_positive+true_negative)/(true_positive+true_negative+false_positive+false_negative)

In [34]:
print(recall)
print(accuracy)

print(true_positive)
print(true_negative) # Not trivial predicted as not trivial
print(false_negative)
print(false_positive) # Not trivial predicted as trivial

0.6666666666666666
0.8372093023255814
8
28
4
3


In [35]:
# However in the context of this case we care about how many were classified correct over all classified correctly + those we flag as trivial but actually not trivial

recall_contextualised = true_negative/(true_negative+false_positive)
print(f'Contextualised recall:{recall_contextualised}')

Contextualised recall:0.9032258064516129


In [36]:
predicted_vs_output[(predicted_vs_output['target']==False) & (predicted_vs_output['prediction']==True)]

,text,target,prediction,category
41,"[CLS] Hello \nsorry to bother you, is this Kev...",False,True,Unsure
48,[CLS] nan [SEP],False,True,Scam
0,[CLS] nan [SEP],False,True,Info/News/Opinion


In [37]:
predicted_vs_output[(predicted_vs_output['text']=='[CLS] nan [SEP]')]

,text,target,prediction,category
48,[CLS] nan [SEP],False,True,Scam
0,[CLS] nan [SEP],False,True,Info/News/Opinion


Noted that the recall can be increased to 96% if we automatically classify any nan sentences as non trivial.
- Index 48 is an image
- Index 0 is an null

Identified 8 out of 12 trivial messages

## Extra testing using spam dataset

In [43]:
spam_df = pd.read_excel('../../src/data/spam.xlsx')

In [44]:
spam_df = spam_df.iloc[:, 1:3]
spam_df.columns = ['text', 'is_trivial']

In [45]:
print(spam_df)

                                                   text  is_trivial
0     Go until jurong point, crazy.. Available only ...           1
1                         Ok lar... Joking wif u oni...           1
2     U dun say so early hor... U c already then say...           1
3     Nah I don't think he goes to usf, he lives aro...           1
4     Even my brother is not like to speak with me. ...           1
...                                                 ...         ...
5566  Want explicit SEX in 30 secs? Ring 02073162414...           0
5567  ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...           0
5568  Had your contract mobile 11 Mnths? Latest Moto...           0
5569  REMINDER FROM O2: To get 2.50 pounds free call...           0
5570  This is the 2nd time we have tried 2 contact u...           0

[5571 rows x 2 columns]


In [46]:
# Prepping data
spam_df.dropna()
# Not all sentences are imported as strings
spam_df['text'] = spam_df['text'].astype('str')

In [47]:
spam_df['text'] = (spam_df['text'].apply(lambda x: "[CLS] " + x + " [SEP]"))
print(spam_df)
spam_df['token_ids'] = spam_df['text'].apply(lambda x: tokenizer.encode(x, truncation=True))
print(spam_df)

                                                   text  is_trivial
0     [CLS] Go until jurong point, crazy.. Available...           1
1             [CLS] Ok lar... Joking wif u oni... [SEP]           1
2     [CLS] U dun say so early hor... U c already th...           1
3     [CLS] Nah I don't think he goes to usf, he liv...           1
4     [CLS] Even my brother is not like to speak wit...           1
...                                                 ...         ...
5566  [CLS] Want explicit SEX in 30 secs? Ring 02073...           0
5567  [CLS] ASKED 3MOBILE IF 0870 CHATLINES INCLU IN...           0
5568  [CLS] Had your contract mobile 11 Mnths? Lates...           0
5569  [CLS] REMINDER FROM O2: To get 2.50 pounds fre...           0
5570  [CLS] This is the 2nd time we have tried 2 con...           0

[5571 rows x 2 columns]
                                                   text  is_trivial  \
0     [CLS] Go until jurong point, crazy.. Available...           1   
1             [CL

In [48]:
spam_df['segment_ids'] = (spam_df['token_ids'].apply(lambda x: [1] * len(x)))

In [49]:
spam_df['tensor_token'] = spam_df.apply(lambda x: torch.tensor([x['token_ids']]), axis=1)
spam_df['tensor_segment'] = spam_df.apply(lambda x: torch.tensor([x['segment_ids']]), axis=1)


In [50]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    spam_df['hidden_states'] = spam_df.apply(lambda x: model(x['tensor_token'],x['tensor_segment'])[2], axis=1)


In [51]:
spam_df['hidden_states'].apply(lambda x: print(x[-2][0].shape))


torch.Size([36, 768])
torch.Size([19, 768])
torch.Size([22, 768])
torch.Size([21, 768])
torch.Size([22, 768])
torch.Size([52, 768])
torch.Size([34, 768])
torch.Size([50, 768])
torch.Size([14, 768])
torch.Size([16, 768])
torch.Size([28, 768])
torch.Size([25, 768])
torch.Size([13, 768])
torch.Size([15, 768])
torch.Size([24, 768])
torch.Size([32, 768])
torch.Size([21, 768])
torch.Size([42, 768])
torch.Size([11, 768])
torch.Size([40, 768])
torch.Size([30, 768])
torch.Size([22, 768])
torch.Size([37, 768])
torch.Size([55, 768])
torch.Size([11, 768])
torch.Size([25, 768])
torch.Size([44, 768])
torch.Size([19, 768])
torch.Size([12, 768])
torch.Size([15, 768])
torch.Size([46, 768])
torch.Size([27, 768])
torch.Size([52, 768])
torch.Size([9, 768])
torch.Size([28, 768])
torch.Size([13, 768])
torch.Size([18, 768])
torch.Size([11, 768])
torch.Size([21, 768])
torch.Size([45, 768])
torch.Size([19, 768])
torch.Size([39, 768])
torch.Size([23, 768])
torch.Size([87, 768])
torch.Size([28, 768])
torch.Size(

0       None
1       None
2       None
3       None
4       None
        ... 
5566    None
5567    None
5568    None
5569    None
5570    None
Name: hidden_states, Length: 5571, dtype: object

In [53]:
spam_df['sentence_embedding'] = spam_df['hidden_states'].apply(lambda x: torch.mean(x[-2][0], dim=0))

In [54]:
spam_df['prediction'] = spam_df['sentence_embedding'].apply(lambda x: classify_trivial(x))

In [55]:
predicted_vs_output = pd.concat([spam_df['text'], spam_df['is_trivial'], spam_df['prediction']], axis=1)
# predicted_vs_output = predicted_vs_output.rename(columns={'is_trivial':'target', 'text':'prediction'})
predicted_vs_output.columns = ['text', 'target','prediction']

print(predicted_vs_output)

                                                   text  target  prediction
0     [CLS] Go until jurong point, crazy.. Available...       1       False
1             [CLS] Ok lar... Joking wif u oni... [SEP]       1        True
2     [CLS] U dun say so early hor... U c already th...       1        True
3     [CLS] Nah I don't think he goes to usf, he liv...       1        True
4     [CLS] Even my brother is not like to speak wit...       1        True
...                                                 ...     ...         ...
5566  [CLS] Want explicit SEX in 30 secs? Ring 02073...       0       False
5567  [CLS] ASKED 3MOBILE IF 0870 CHATLINES INCLU IN...       0       False
5568  [CLS] Had your contract mobile 11 Mnths? Lates...       0       False
5569  [CLS] REMINDER FROM O2: To get 2.50 pounds fre...       0       False
5570  [CLS] This is the 2nd time we have tried 2 con...       0       False

[5571 rows x 3 columns]


In [56]:
# Calculate the recall of this methodology
true_positive = len(predicted_vs_output[(predicted_vs_output['target']==True) & (predicted_vs_output['prediction']==True)])
true_negative = len(predicted_vs_output[(predicted_vs_output['target']==False) & (predicted_vs_output['prediction']==False)])

false_negative = len(predicted_vs_output[(predicted_vs_output['target']==True) & (predicted_vs_output['prediction']==False)])
false_positive = len(predicted_vs_output[(predicted_vs_output['target']==False) & (predicted_vs_output['prediction']==True)])

recall = true_positive/(true_positive+false_negative)
accuracy = (true_positive+true_negative)/(true_positive+true_negative+false_positive+false_negative)

In [57]:
print(recall)
print(accuracy)

print(true_positive)
print(true_negative) # Not trivial predicted as not trivial
print(false_negative)
print(false_positive) # Not trivial predicted as trivial

0.5893228084752804
0.6440495422724825
2837
751
1977
6


In [58]:
# However in the context of this case we care about how many were classified correct over all classified correctly + those we flag as trivial but actually not trivial

recall_contextualised = true_negative/(true_negative+false_positive)
print(f'Contextualised recall:{recall_contextualised}')

Contextualised recall:0.9920739762219286


In [59]:
print(predicted_vs_output[(predicted_vs_output['target']==False) & (predicted_vs_output['prediction']==True)])

                                                   text  target  prediction
4973  [CLS] Filthy stories and GIRLS waiting for you...       0        True
5029  [CLS] For sale - arsenal dartboard. Good condi...       0        True
5034  [CLS] Bought one ringtone and now getting text...       0        True
5193              [CLS] FROM 88066 LOST Â£12 HELP [SEP]       0        True
5432  [CLS] Money i have won wining number 946 wot d...       0        True
5552  [CLS] Latest News! Police station toilet stole...       0        True


In [60]:
print(predicted_vs_output[(predicted_vs_output['target']==True) & (predicted_vs_output['prediction']==False)].head(10))

                                                 text  target  prediction
0   [CLS] Go until jurong point, crazy.. Available...       1       False
6   [CLS] I'm gonna be home soon and i don't want ...       1       False
7   [CLS] I've been searching for the right words ...       1       False
14  [CLS] So Ã_ pay first lar... Then when is da s...       1       False
15  [CLS] Aft i finish my lunch then i go str down...       1       False
16  [CLS] Ffffffffff. Alright no way I can meet up...       1       False
17  [CLS] Just forced myself to eat a slice. I'm r...       1       False
19  [CLS] Did you catch the bus ? Are you frying a...       1       False
20  [CLS] I'm back &amp; we're packing the car now...       1       False
22  [CLS] Wait that's still not all that clear, we...       1       False


## Rerun model with a mix of checkmate and noncheckmate data

In [65]:
checkmate_messages_df.keys()

Index(['text', 'taggedCategory', 'type', 'isMachineCategorised', 'truthScore',
       'is_trivial', 'token_ids', 'segment_ids', 'tensor_token',
       'tensor_segment', 'hidden_states', 'sentence_embedding'],
      dtype='object')

In [66]:
df_comb = pd.concat([checkmate_messages_df, spam_df], axis=0)
print(df_comb.head(10))

                                                text     taggedCategory  \
0                                    [CLS] nan [SEP]  Info/News/Opinion   
1  [CLS] https://www.mas.gov.sg/news/media-releas...  Info/News/Opinion   
2  [CLS] TN 95546718362782 is out for del. Allow ...         Legitimate   
3  [CLS] 🚩🚩🚩 *"You flag, we check"* 🔍🔍🔍\n\nNot su...  Info/News/Opinion   
4  [CLS] https://form.gov.sg/63f594b42413ea001183...         Legitimate   
5  [CLS] 1. National School Games (NSG) 2023 (Sec...         Legitimate   
6                [CLS] Well done CheckMate 👏👏👏 [SEP]            Trivial   
7  [CLS] https://forceforgood.insead.edu/give\n\n...             Unsure   
8                                    [CLS] nan [SEP]         Legitimate   
9  [CLS] https://www.8world.com/singapore/covid-1...  Info/News/Opinion   

    type isMachineCategorised  truthScore  is_trivial  \
0  image                False    6.853846           0   
1   text                False    5.000000           0   
2  

In [68]:
df_comb['taggedCategory']

0       Info/News/Opinion
1       Info/News/Opinion
2              Legitimate
3       Info/News/Opinion
4              Legitimate
              ...        
5566                  NaN
5567                  NaN
5568                  NaN
5569                  NaN
5570                  NaN
Name: taggedCategory, Length: 5657, dtype: object

In [69]:
df_consol = df_comb.drop(['taggedCategory', 'isMachineCategorised','truthScore','prediction'], axis='columns')

In [73]:
print(df_consol['is_trivial'].unique())

[0 1]


In [80]:
df_train, df_test= train_test_split(
        df_consol, test_size=0.80, random_state=0, stratify=df_consol['is_trivial'])

# Train using 20% and Test using 80%
print(len(df_train))
print(len(df_test))

1131
4526


In [81]:
# average the sentence embeddings of non trivial and trivial messages
df_train_not_trivial = df_train[df_train['is_trivial'] == False]
df_train_trivial = df_train[df_train['is_trivial'] == True]

In [82]:
tensor_train_not_trivial = torch.stack([t for t in df_train_not_trivial['sentence_embedding']])
tensor_train_trivial = torch.stack([t for t in df_train_trivial['sentence_embedding']])


In [83]:
rep_not_trivial = torch.mean(tensor_train_not_trivial, dim=0)
rep_trivial = torch.mean(tensor_train_trivial, dim=0)

In [84]:
def classify_trivial(v):
    distance_from_not_trivial = cosine(rep_not_trivial, v)
    distance_from_trivial = cosine(rep_trivial, v)
    if distance_from_trivial > distance_from_not_trivial:
        return False
    else: 
        return True

In [85]:
df_test['prediction'] = df_test['sentence_embedding'].apply(lambda x: classify_trivial(x))

In [87]:
predicted_vs_output = pd.concat([df_test['text'], df_test['is_trivial'], df_test['prediction']], axis=1)
# predicted_vs_output = predicted_vs_output.rename(columns={'is_trivial':'target', 'text':'prediction'})
predicted_vs_output.columns = ['text', 'target','prediction']

print(predicted_vs_output)

                                                   text  target  prediction
3027  [CLS] You said to me before i went back to bed...       1        True
4136  [CLS] Yup... Hey then one day on fri we can as...       1        True
3531  [CLS] No no. I will check all rooms befor acti...       1        True
2833                         [CLS] Thanx a lot... [SEP]       1        True
1702  [CLS] Sorry, I'll call later in meeting any th...       1       False
...                                                 ...     ...         ...
880   [CLS] naughty little thought: 'its better to f...       1        True
4623  [CLS] And of course you should make a stink! [...       1        True
4790  [CLS] Hey chief, can you give me a bell when y...       1        True
2931      [CLS] Just finished. Missing you plenty [SEP]       1        True
3873  [CLS] Y cant u try new invention to fly..i'm n...       1        True

[4526 rows x 3 columns]


In [92]:
# Calculate the recall of this methodology
true_positive = len(predicted_vs_output[(predicted_vs_output['target']==True) & (predicted_vs_output['prediction']==True)])
true_negative = len(predicted_vs_output[(predicted_vs_output['target']==False) & (predicted_vs_output['prediction']==False)])

false_negative = len(predicted_vs_output[(predicted_vs_output['target']==True) & (predicted_vs_output['prediction']==False)])
false_positive = len(predicted_vs_output[(predicted_vs_output['target']==False) & (predicted_vs_output['prediction']==True)])

recall = true_positive/(true_positive+false_negative)
accuracy = (true_positive+true_negative)/(true_positive+true_negative+false_positive+false_negative)

In [93]:
print(recall)
print(accuracy)

print(true_positive)
print(true_negative) # Not trivial predicted as not trivial
print(false_negative)
print(false_positive) # Not trivial predicted as trivial

0.9697674418604652
0.968625718073354
3753
631
117
25


In [94]:
# However in the context of this case we care about how many were classified correct over all classified correctly + those we flag as trivial but actually not trivial

recall_contextualised = true_negative/(true_negative+false_positive)
print(f'Contextualised recall:{recall_contextualised}')

Contextualised recall:0.961890243902439


Using 20% of the dataset to create the representative vector, we were able to create a model that has an accuracy of 96.8%

Contextualise Recall = Correctly Classfied Non-Trivial Messages/(Correctly Classified Non-Trivial Messages + Non-trivial messages classified as trivial message)
96.2%